In [204]:
#Import Required Libraries
import pandas as pd
import numpy as np
import requests

In [205]:
#Code Scrape Wiki Page
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki1 = requests.get(wiki)
#Ignore 'Not assigned' Boroughs
wiki2 = pd.read_html(wiki1.content, header = 0)[0]
df = wiki2[wiki2.Borough != 'Not assigned']
df.reset_index(inplace = True)


In [206]:
# Group different neighborhoods seperated by a comma
wiki_grouped = df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))

In [207]:
#assign a neighborhood to another
mask = wiki_grouped['Neighbourhood'] == "Not assigned"
wiki_grouped.loc[mask, 'Neighbourhood'] = wiki_grouped.loc[mask, 'Borough']

In [208]:
#Shape of wiki_grouped
print(f" {wiki_grouped.head()}")
print(wiki_grouped.shape)

   Postal Code      Borough                           Neighbourhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae
(103, 3)


In [209]:
##Part2
#stored csv file
url = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [210]:
#join both dataframes
df = df.join(df_geo.set_index('Postal Code'), on='Postal Code')
df.drop(columns=['index'])
df.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [211]:
##Part 3
!pip install geopy
from geopy.geocoders import Nominatim
! pip install folium==0.5.0
import folium

In [214]:
#assign variables to functions
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [243]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='Stamen Terrain')

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = folium.Popup(f"{neighborhood},{borough}", parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        ).add_to(map_Toronto)  
    
map_Toronto